In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

#openAiContentSafetyConnectionName

ai_project_conn_str = os.getenv("AZURE_LOCATION")+".api.azureml.ms;"+os.getenv("AZURE_SUBSCRIPTION_ID")+";"+os.getenv("AZURE_RESOURCE_GROUP")+";"+os.getenv("AZURE_AI_PROJECT_NAME")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=ai_project_conn_str,
)

In [4]:
from azure.ai.inference.prompts import PromptTemplate

prompt_template = PromptTemplate.from_prompty(file_path=os.getcwd() + "/researcher.prompty")


instructions = "Can you find the latest camping trends and what folks are doing in the winter?"
feedback= 'None'
messages = prompt_template.create_messages(instructions=instructions, feedback=feedback)

In [5]:
bing_connection = project_client.connections.get(
    connection_name='agentbingsearch'
)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

prompt_template = PromptTemplate.from_prompty(file_path=os.getcwd() + "/researcher.prompty")

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model="gpt-4",
        name="my-assistant",
        instructions=messages[0]['content'],
        tools=bing.definitions,
    )

    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="""Can you find the latest camping trends and what folks are doing in the winter?"
        
        """,
    )
    print(f"Created message, ID: {message.id}")

    # # Create and process agent run in thread with tools
    # run = project_client.agents.create_stream(thread_id=thread.id, assistant_id=agent.id)

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Retrieve run step details to get Bing Search query link
    # To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']
    print(f"Last run step detail: {run_steps_data}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")
    print(messages.data[0]['content'][0]['text']['value'])

Created agent, ID: asst_018XlKcMl54zowbo5mRQOxm6
Created thread, ID: thread_DSK9FDKWMFoinwxYEEPpCE8w
Created message, ID: msg_t84ImYwOL3jpQtR5iszZlIja
Run finished with status: RunStatus.COMPLETED
Last run step detail: [{'id': 'step_OCMvL4M2xJNebq6sYw5Aoo80', 'object': 'thread.run.step', 'created_at': 1739372520, 'run_id': 'run_yspJOh6xBiuhsoSAXqilKDdJ', 'assistant_id': 'asst_018XlKcMl54zowbo5mRQOxm6', 'thread_id': 'thread_DSK9FDKWMFoinwxYEEPpCE8w', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1739372533, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_jJeRKVJNkQmJjn2h6kKlGDKu'}}, 'usage': {'prompt_tokens': 2394, 'completion_tokens': 468, 'total_tokens': 2862, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_zvTq6ogVePagtkgRKVDzGkHp', 'object': 'thread.run.step', 'created_at': 1739372520, 'run_id': 'run_yspJOh6xBiuhsoSAXqilKDdJ', 'assis

In [ ]:
research = messages.data[0]['content'][0]['text']['value']
import json 

r = json.loads(research)
r['results']